# KBO 선수 데이터 가져오기
```
데이터 출저 : http://www.statiz.co.kr
사용 데이터 : 1983 ~ 1988 / 2015 ~ 2020

```

In [51]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

import re

import time
from tqdm import tqdm

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.by import By


In [2]:
# 83-88년도 모든 투수 데이터 가져오기
url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=1983&ye=1988&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=480&si=&cn='


In [3]:
response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')


In [4]:
records = []
birth_list = []

skip_row = 1
for tr in soup.find_all("tr")[1:]:
    if tr.find_all("th"):
        skip_row+=1
        continue
    tmp = []
    cnt=0
    flag_birth = True
    for text in tr.find_all("td"):
        if cnt == 3:
            cnt+=1
            continue
            
        add_text = '0' if (text.text.replace(' ', '') == '') else text.text

        try:
            tmp.append(float(add_text))
        except:
            tmp.append(add_text)
        cnt+=1
        
        #선수 생년월일 데이터
        try:
            if flag_birth:    
                birth_list.append(text.find("a")["href"][-10:])
                flag_birth = False
        except:
            pass
        
    if '질문/건의' in str(tmp[0]):
        break
    records.append(tmp)
    skip_row+=1

In [5]:
rank = []; name = []; team =[]; gp = []; cg = []; sho = []; gs = []; win = []; loss = [];
save = []; hold = []; inning = []; runs = []; earned_run = []; batter_faced = [];
hits = []; base2 = []; base3 = []; home_run = []; walk = []; intentional_walk = [];
hb = []; so = []; balk=[]; wild =[]; ERA = []; FIP = []; WHIP = []; ERAplus=[]; FIPplus=[]; WAR = []; WPA = []

In [6]:
status_list = [rank, name, team, gp, cg, sho, gs, win, loss, save, hold, inning, runs, earned_run, batter_faced, hits, base2, base3, home_run, walk, intentional_walk, hb, so, balk, wild, ERA, FIP, WHIP, ERAplus, FIPplus, WAR, WPA]

In [7]:
len(records), len(status_list)

(452, 32)

In [8]:
for record in records:
    for idx, status in enumerate(status_list):
        if idx == len(records)-1:
            break
        status.append(record[idx])

In [9]:
team_dict = {
    'M' : 'MBC Blue Dragons',
    'O' : 'OB Bears',
    '롯' : 'Lotte Giants',
    '빙' : 'Binggre Eagles',
    '삼' : 'Samsung Lions',
    '청' : 'Chungbo Pintos',
    '태' : 'Pacific Dolphins',
    '해' : 'Haitai Tigers',
    '해빙' : 'Haitai Tigers Binggre Eagles'
}

In [10]:
year = []
team_name = []

r = re.compile("[0-9]+")

for t in team:
    year.append('19'+r.findall(t)[0])
    team_name.append(team_dict[r.split(t)[1]])

In [11]:
set(team_name)

{'Binggre Eagles',
 'Chungbo Pintos',
 'Haitai Tigers',
 'Haitai Tigers Binggre Eagles',
 'Lotte Giants',
 'MBC Blue Dragons',
 'OB Bears',
 'Pacific Dolphins',
 'Samsung Lions'}

In [12]:
birth_list = birth_list[:-1]

In [13]:
df_8388 = pd.DataFrame({
    'rank' : rank,
    'name' : name,
    'birth' : birth_list,
    'team' : team_name,
    'year' : year,
    'gp' : gp,
    'cg' : cg,
    'sho' : sho,
    'gs' : gs,
    'win' : win,
    'loss' : loss,
    'save' : save,
    'hold' : hold,
    'inning' : inning,
    'runs' : runs,
    'earned_run' : earned_run,
    'batter_faced' : batter_faced,
    'hits' : hits,
    'base2' : base2,
    'base3' : base3,
    'home_run' : home_run,
    'walk' : walk,
    'intentional_walk' : intentional_walk,
    'hb' : hb,
    'so' : so,
    'balk' : balk,
    'wild' : wild,
    'ERA' : ERA,
    'FIP' : FIP,
    'WHIP' : WHIP,
    'ERAplus' : ERAplus,
    'FIPplus' : FIPplus,
    'WAR' : WAR,
    'WPA' : WPA
})

In [14]:
df_8388.head()

,rank,name,birth,team,year,gp,cg,sho,gs,win,...,so,balk,wild,ERA,FIP,WHIP,ERAplus,FIPplus,WAR,WPA
0,1.0,선동열,1963-01-10,Haitai Tigers,1986,39.0,19.0,8.0,22.0,24.0,...,214.0,2.0,3.0,0.99,1.16,0.78,307.1,262.7,14.89,0.0
1,2.0,장명부,1950-12-27,Samsung Lions,1983,60.0,36.0,5.0,44.0,30.0,...,220.0,1.0,6.0,2.34,2.42,1.16,143.4,138.5,13.63,0.0
2,3.0,선동열,1963-01-10,Haitai Tigers,1988,31.0,9.0,1.0,12.0,16.0,...,200.0,1.0,0.0,1.21,1.26,0.85,316.2,303.8,11.79,0.0
3,4.0,최동원,1958-05-24,Lotte Giants,1986,39.0,17.0,4.0,21.0,19.0,...,208.0,0.0,2.0,1.55,1.49,0.97,191.8,200.0,11.74,0.0
4,5.0,김시진,1958-03-20,Samsung Lions,1985,47.0,10.0,2.0,29.0,25.0,...,201.0,1.0,10.0,2.00,2.59,1.09,177.3,137.0,10.52,0.0


In [15]:
df_8388.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              452 non-null    float64
 1   name              452 non-null    object 
 2   birth             452 non-null    object 
 3   team              452 non-null    object 
 4   year              452 non-null    object 
 5   gp                452 non-null    float64
 6   cg                452 non-null    float64
 7   sho               452 non-null    float64
 8   gs                452 non-null    float64
 9   win               452 non-null    float64
 10  loss              452 non-null    float64
 11  save              452 non-null    float64
 12  hold              452 non-null    float64
 13  inning            452 non-null    float64
 14  runs              452 non-null    float64
 15  earned_run        452 non-null    float64
 16  batter_faced      452 non-null    float64
 1

In [16]:
# 구원 투수 정보 가져오기 - 셀레니움

url_Relief = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=1983&ye=1988&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=10&o1=WAR_R&de=1&o2=WAR&lr=0&tr=&cv=&ml=1&sn=500&si=&cn='

driver = webdriver.Chrome(executable_path='../../driver/chromedriver')
driver.get(url=url_Relief)

/var/folders/jt/jw2pb8hx71d48w0rx0cgqf_80000gn/T/ipykernel_27638/1356277571.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../../driver/chromedriver')


In [17]:
relief_text = (driver.find_elements(By.ID, 'fixcol')[0].text)

driver.close()

In [18]:
relief_text = relief_text.replace('\n', ' ')
relief_text = relief_text.replace('순 이름 팀 정렬 구원WAR ', '')

In [19]:
relief_list = []
tmp = []
cnt = 0
for r_text in relief_text.split():
    tmp.append(r_text)
    cnt += 1
    try:
        if cnt == 4:
            float(r_text) #구원WAR 데이터가 없으면 에러 발생
            relief_list.append(tmp)
            tmp = []
            cnt = 0
    except:
        break

In [20]:
relief_year = []

r = re.compile("[0-9]+")

for t in relief_list:
    relief_year.append('19'+r.findall(t[2])[0])

In [24]:
idx_list = []
for i in range(len(relief_list)):
    idx_list.append(int(df_8388[(df_8388['name'] == relief_list[i][1]) & (df_8388['year'] == relief_year[i])].index.values[0]))
    

In [25]:
len(idx_list), len(df_8388)

(435, 452)

In [26]:
df_8388['relief_WAR'] = np.nan

for idx in idx_list:
    try:
        df_8388['relief_WAR'].iloc[idx] =float(relief_list[idx][3])
    except:
        pass

/Users/seokholee/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [27]:
df_8388.head()

,rank,name,birth,team,year,gp,cg,sho,gs,win,...,balk,wild,ERA,FIP,WHIP,ERAplus,FIPplus,WAR,WPA,relief_WAR
0,1.0,선동열,1963-01-10,Haitai Tigers,1986,39.0,19.0,8.0,22.0,24.0,...,2.0,3.0,0.99,1.16,0.78,307.1,262.7,14.89,0.0,6.18
1,2.0,장명부,1950-12-27,Samsung Lions,1983,60.0,36.0,5.0,44.0,30.0,...,1.0,6.0,2.34,2.42,1.16,143.4,138.5,13.63,0.0,5.68
2,3.0,선동열,1963-01-10,Haitai Tigers,1988,31.0,9.0,1.0,12.0,16.0,...,1.0,0.0,1.21,1.26,0.85,316.2,303.8,11.79,0.0,4.89
3,4.0,최동원,1958-05-24,Lotte Giants,1986,39.0,17.0,4.0,21.0,19.0,...,0.0,2.0,1.55,1.49,0.97,191.8,200.0,11.74,0.0,4.64
4,5.0,김시진,1958-03-20,Samsung Lions,1985,47.0,10.0,2.0,29.0,25.0,...,1.0,10.0,2.00,2.59,1.09,177.3,137.0,10.52,0.0,3.97


In [28]:
df_8388.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              452 non-null    float64
 1   name              452 non-null    object 
 2   birth             452 non-null    object 
 3   team              452 non-null    object 
 4   year              452 non-null    object 
 5   gp                452 non-null    float64
 6   cg                452 non-null    float64
 7   sho               452 non-null    float64
 8   gs                452 non-null    float64
 9   win               452 non-null    float64
 10  loss              452 non-null    float64
 11  save              452 non-null    float64
 12  hold              452 non-null    float64
 13  inning            452 non-null    float64
 14  runs              452 non-null    float64
 15  earned_run        452 non-null    float64
 16  batter_faced      452 non-null    float64
 1

In [29]:
df_8388.to_csv('1983-1988년 투수 데이터.csv')

In [30]:
# 15-20년도 모든 투수 데이터 가져오기
url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=2015&ye=2020&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=1550&si=&cn='

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup_1520 = BeautifulSoup(html, 'html.parser')




In [31]:
records_1520 = []
birth_list = []

skip_row = 1
for tr in soup_1520.find_all("tr")[1:]:
    if tr.find_all("th"):
        skip_row+=1
        continue
    tmp = []
    cnt=0
    flag_birth = True
    for text in tr.find_all("td"):
        if cnt == 3:
            cnt+=1
            continue
            
        add_text = '0' if (text.text.replace(' ', '') == '') else text.text

        try:
            tmp.append(float(add_text))
        except:
            tmp.append(add_text)
        cnt+=1
        
        #선수 생년월일 데이터
        try:
            if flag_birth:    
                birth_list.append(text.find("a")["href"][-10:])
                flag_birth = False
        except:
            pass
        
    if '질문/건의' in str(tmp[0]):
        break
    records_1520.append(tmp)
    skip_row+=1

In [32]:
rank = []; name = []; team =[]; gp = []; cg = []; sho = []; gs = []; win = []; loss = [];
save = []; hold = []; inning = []; runs = []; earned_run = []; batter_faced = [];
hits = []; base2 = []; base3 = []; home_run = []; walk = []; intentional_walk = [];
hb = []; so = []; balk=[]; wild =[]; ERA = []; FIP = []; WHIP = []; ERAplus=[]; FIPplus=[]; WAR = []; WPA = []

In [33]:
status_list = [rank, name, team, gp, cg, sho, gs, win, loss, save, hold, inning, runs, earned_run, batter_faced, hits, base2, base3, home_run, walk, intentional_walk, hb, so, balk, wild, ERA, FIP, WHIP, ERAplus, FIPplus, WAR, WPA]

In [34]:
len(records_1520), len(status_list)

(1533, 32)

In [35]:
birth_list = birth_list[:-1]

In [36]:
for record_1520 in records_1520:
    for idx, status in enumerate(status_list):
        if idx == len(records)-1:
            break
        status.append(record_1520[idx])

In [37]:
team_dict_1520 = {
    '넥' : '넥센 히어로즈 ',
    '두' : '두산 베어스 ',
    '롯' : '롯데 자이언츠 ',
    '삼' : '삼성 라이온즈 ',
    '한' : '한화 이글스 ',
    'K' : 'KIA 타이거즈 ',
    'L' : 'LG 트윈스 ',
    'N' : 'NC 다이노스 ',
    'S' : 'SK 와이번스 ',
    'k' : 'kt wiz ',
    '키' : '키움 히어로즈'
}

In [38]:
team_names = []
r = re.compile("[0-9]+")
tmp = r.split(team[-2])[1]
for k, v in team_dict_1520.items():    
    tmp = tmp.replace(k,v)
tmp = (re.sub("\s$", '', tmp))


In [39]:
year = []
team_name = []

r = re.compile("[0-9]+")

for t in team:
    year.append('20'+r.findall(t)[0])
    
    tmp = r.split(t)[1]
    for k, v in team_dict_1520.items():    
        tmp = tmp.replace(k,v)
    team_name.append(re.sub("\s$", '', tmp))

In [40]:
df_1520 = pd.DataFrame({
    'rank' : rank,
    'name' : name,
    'birth' : birth_list,
    'team' : team_name,
    'year' : year,
    'gp' : gp,
    'cg' : cg,
    'sho' : sho,
    'gs' : gs,
    'win' : win,
    'loss' : loss,
    'save' : save,
    'hold' : hold,
    'inning' : inning,
    'runs' : runs,
    'earned_run' : earned_run,
    'batter_faced' : batter_faced,
    'hits' : hits,
    'base2' : base2,
    'base3' : base3,
    'home_run' : home_run,
    'walk' : walk,
    'intentional_walk' : intentional_walk,
    'hb' : hb,
    'so' : so,
    'balk' : balk,
    'wild' : wild,
    'ERA' : ERA,
    'FIP' : FIP,
    'WHIP' : WHIP,
    'ERAplus' : ERAplus,
    'FIPplus' : FIPplus,
    'WAR' : WAR,
    'WPA' : WPA
})

In [41]:
df_1520.head()

,rank,name,birth,team,year,gp,cg,sho,gs,win,...,so,balk,wild,ERA,FIP,WHIP,ERAplus,FIPplus,WAR,WPA
0,1.0,알칸타라,1992-12-04,두산 베어스,2020,31.0,1.0,0.0,31.0,20.0,...,182.0,1.0,3.0,2.54,3.10,1.03,182.6,149.7,8.31,4.35
1,2.0,양현종,1988-03-01,KIA 타이거즈,2015,32.0,1.0,1.0,31.0,15.0,...,157.0,0.0,6.0,2.44,4.57,1.24,202.1,108.5,8.14,3.94
2,3.0,스트레일리,1988-12-01,롯데 자이언츠,2020,31.0,0.0,0.0,31.0,15.0,...,205.0,0.0,4.0,2.50,3.01,1.02,195.1,159.0,7.53,3.84
3,4.0,양현종,1988-03-01,KIA 타이거즈,2019,29.0,2.0,2.0,29.0,16.0,...,163.0,0.0,1.0,2.29,2.63,1.07,183.1,160.3,7.35,3.81
4,5.0,브룩스,1990-04-27,KIA 타이거즈,2020,23.0,1.0,1.0,23.0,11.0,...,130.0,1.0,3.0,2.50,2.72,1.02,192.2,177.8,7.16,3.39


In [42]:
# 15-20 구원 투수 정보 가져오기 - 셀레니움

url_Relief = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=2015&ye=2020&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=10&o1=WAR_R&de=1&o2=WAR&lr=0&tr=&cv=&ml=1&sn=1550&si=&cn='

driver = webdriver.Chrome(executable_path='../../driver/chromedriver')
driver.get(url=url_Relief)

/var/folders/jt/jw2pb8hx71d48w0rx0cgqf_80000gn/T/ipykernel_27638/2089346043.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../../driver/chromedriver')


In [43]:
relief_text = (driver.find_elements(By.ID, 'fixcol')[0].text)

driver.close()




In [44]:
relief_text = relief_text.replace('\n', ' ')
relief_text = relief_text.replace('순 이름 팀 정렬 구원WAR ', '')

In [45]:
relief_list = []
tmp = []
cnt = 0
for r_text in relief_text.split():
    tmp.append(r_text)
    cnt += 1
    try:
        if cnt == 4:
            float(r_text) #구원WAR 데이터가 없으면 에러 발생
            relief_list.append(tmp)
            tmp = []
            cnt = 0
    except:
        break

In [47]:
relief_list[:5]

[['1', '손승락', '17롯', '4.12'],
 ['2', '고우석', '19L', '3.89'],
 ['3', '윤석민', '15K', '3.87'],
 ['4', '정우람', '19한', '3.82'],
 ['5', '조상우', '15넥', '3.36']]

In [48]:
relief_year = []
idx_list = []

r = re.compile("[0-9]+")

for t in relief_list:
    relief_year.append('20'+r.findall(t[2])[0])
    
for i in range(len(relief_list)):
    idx_list.append(int(df_1520[(df_1520['name'] == relief_list[i][1]) & (df_1520['year'] == relief_year[i])].index.values[0]))
    

In [49]:
df_1520['relief_WAR'] = np.nan

for idx in idx_list:
    try:
        df_1520['relief_WAR'].iloc[idx] =float(relief_list[idx][3])
    except:
        pass

/Users/seokholee/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [50]:
df_1520.head()

,rank,name,birth,team,year,gp,cg,sho,gs,win,...,balk,wild,ERA,FIP,WHIP,ERAplus,FIPplus,WAR,WPA,relief_WAR
0,1.0,알칸타라,1992-12-04,두산 베어스,2020,31.0,1.0,0.0,31.0,20.0,...,1.0,3.0,2.54,3.10,1.03,182.6,149.7,8.31,4.35,NaN
1,2.0,양현종,1988-03-01,KIA 타이거즈,2015,32.0,1.0,1.0,31.0,15.0,...,0.0,6.0,2.44,4.57,1.24,202.1,108.5,8.14,3.94,3.89
2,3.0,스트레일리,1988-12-01,롯데 자이언츠,2020,31.0,0.0,0.0,31.0,15.0,...,0.0,4.0,2.50,3.01,1.02,195.1,159.0,7.53,3.84,NaN
3,4.0,양현종,1988-03-01,KIA 타이거즈,2019,29.0,2.0,2.0,29.0,16.0,...,0.0,1.0,2.29,2.63,1.07,183.1,160.3,7.35,3.81,NaN
4,5.0,브룩스,1990-04-27,KIA 타이거즈,2020,23.0,1.0,1.0,23.0,11.0,...,1.0,3.0,2.50,2.72,1.02,192.2,177.8,7.16,3.39,NaN


## 연봉 데이터 가져오기

### 83 - 88년 연봉 데이터 가져오기
- 90년도 까지 연봉 데이터가 있다면 가져오기

In [84]:
pitcher_to_salary = []
year = []
salary = []


cnt = 0
for idx in tqdm(range(len(df_8388['name']))):
    url = 'http://www.statiz.co.kr/player.php?opt=11&name=' + df_8388['name'][idx] + '&birth=' + df_8388['birth'][idx]
    
    response = requests.get(url)
    
    if response.status_code == 200:
        html = response.text
        soup_salary = BeautifulSoup(html, 'html.parser')
    else:
        print('!!! Error Request !!!')
    
    if [df_8388['name'][idx], df_8388['birth'][idx]] in pitcher_to_salary:
        continue
        
    pitcher_to_salary.append([df_8388['name'][idx], df_8388['birth'][idx]])
    
    try:
        for s in soup_salary.find('table', class_ = 'table table-striped table-condensed').find_all("tr")[1:]:
            if int(s.find_all("td")[0].text) > 1990:
                break
            
            # 추가할 수 있는 연봉 정보가 있다면 출력시키기 - 출력 정보가 없다면 사용할 수 있는 연봉 정보가 없는 것
            try:
                print(int(s.find_all("td")[1].text.replace(',',''))) #연봉
                print(df_8388['name'][idx], df_8388['birth'][idx])
            except:
                pass
    except:
        pass

    
    time.sleep(0.3)
    

100%|█████████████████████████████████████████| 452/452 [01:46<00:00,  4.24it/s]


### 15-20년 연봉 데이터 가져오기 

In [85]:
pitcher_to_salary = []
year = []
salary = []


for idx in tqdm(range(len(df_1520['name']))):
    url = 'http://www.statiz.co.kr/player.php?opt=11&name=' + df_1520['name'][idx] + '&birth=' + df_1520['birth'][idx]
    
    response = requests.get(url)
    
    if response.status_code == 200:
        html = response.text
        soup_salary = BeautifulSoup(html, 'html.parser')
    else:
        print('!!! Error Request !!!')
    
    if [df_1520['name'][idx], df_1520['birth'][idx]] in pitcher_to_salary:
        continue
        
    pitcher_to_salary.append([df_1520['name'][idx], df_1520['birth'][idx]])
    
    tmp_year = []
    tmp_salary = []
    try:
        for s in soup_salary.find('table', class_ = 'table table-striped table-condensed').find_all("tr")[1:]:
            tmp_year.append(int(s.find_all("td")[0].text)) #년도
            try:
                tmp_salary.append(int(s.find_all("td")[1].text.replace(',',''))) #연봉
            except:
                tmp_salary.append(0)
    except:
        print("Error Pitcher : ", df_1520['name'][idx], '/', birth_list[idx])
        tmp_year.append(0)
        tmp_salary.append(0)
        
    year.append(tmp_year)
    salary.append(tmp_salary)
    
    time.sleep(0.3)

    
    

100%|███████████████████████████████████████| 1533/1533 [06:20<00:00,  4.03it/s]


In [86]:
len(year), len(salary), len(pitcher_to_salary), len(df_1520['name'].unique())

(592, 592, 592, 578)

In [88]:
df_1520 = df_1520.astype({'year':'int'})

In [89]:
add_to_salary_idx = []
salary_to_df = []
for idx1, i in enumerate(year):
    
    flag1 = (df_1520['name'] == pitcher_to_salary[idx1][0]) & (df_1520['birth'] == pitcher_to_salary[idx1][1])
    
    for idx2, y in enumerate(i):
        if y == 0:
            print(pitcher_to_salary[idx1][1])
#             add_to_salary_idx.append(birth_list.index(pitcher_to_salary[idx1][1]))
            continue
            
        flag2 = df_1520['year'] == y
        
        try:
            add_to_salary_idx.append(df_1520[(flag1) & (flag2)].index.values[0])
            salary_to_df.append(salary[idx1][idx2])
        except:
            pass

In [90]:
df_1520 = df_1520.iloc[add_to_salary_idx]

In [91]:
df_1520['salary'] = salary_to_df
df_1520 = df_1520.sort_index()

In [92]:
df_1520[df_1520['name']=='알칸타라']

,rank,name,birth,team,year,gp,cg,sho,gs,win,...,wild,ERA,FIP,WHIP,ERAplus,FIPplus,WAR,WPA,relief_WAR,salary
0,1.0,알칸타라,1992-12-04,두산 베어스,2020,31.0,1.0,0.0,31.0,20.0,...,3.0,2.54,3.10,1.03,182.6,149.7,8.31,4.35,NaN,45000
143,144.0,알칸타라,1992-12-04,KIA 타이거즈,2019,27.0,1.0,0.0,27.0,11.0,...,2.0,4.01,3.98,1.25,103.1,103.3,2.60,0.88,NaN,0


In [93]:
df_1520.to_csv('2015-2020년 투수 데이터.csv')